# 基本功能

pandas中的对象包含多种方法。

首先导入pandas相关的模块。

In [1]:
import pandas as pd
import numpy as np

## reindex

pandas中的数据结构具有的一个重要方法即`reindex`，该方法会接受一个`list`作为参数，用于创建一个与原对象相同的新对象，并将新对象的`index`设置为给定的`list`参数，

In [2]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

`reindex`的`method`属性为`ffill`时，指定了对有序`index`列表中缺省项采用依次填补的方式补充完整。若不指定`method`属性，则新对象的`index`列表的缺省向为`NaN`。

In [3]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [4]:
obj3.reindex(range(6))

0      blue
1       NaN
2    purple
3       NaN
4    yellow
5       NaN
dtype: object

对于`DataFrame`类型，`reindex`方法可以更改`index`、`columns`，或者两者同时修改。

我们还可以通过`loc`方法替代`reindex`方法，更加简洁的完成同样的功能**（官方已经不推荐这一做法，该方法存在安全隐患）**。

In [5]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'], 
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [6]:
states = ['Texas', 'Utah', 'California']
frame2 = frame.reindex(['a', 'b', 'c', 'd'], columns=states)
frame2

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


|参数|备注|
|-|-|
|index||
|method||
|fill_value||
|limit||
|tolerance||
|level||
|copy||

## drop

对于`Series`对象，`drop`方法会删除对象中一行或多行，并返回一个删除行后的新对象，原对象不受影响。

In [7]:
# 删除单行
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [8]:
# 删除多行
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

对于`DataFrame`对象，`drop`方法既可以根据`index`删除行，也可以根据`columns`删除列。在未指定`axis`值时，`drop`方法行为与`Series`对象中类似，默认删除`DataFrame`对象的行，当`axis`值为`1`或`columns`时，`drop`删除`DataFrame`的指定列。

In [9]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [10]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [11]:
data.drop('two', axis='columns')

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


当指定`inplace`为`True`时，`DataFrame`将删除原对象的对应行与列。

In [12]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

## 索引、选择、筛选

`Sereis`对象的索引（`obj[...]`）与NumPy数组类似。

### 索引

In [14]:
obj = pd.Series(np.arange(4.0), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [15]:
obj['b']

1.0

In [16]:
obj[1]

1.0

In [17]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [18]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [19]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [20]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

In [22]:
# 赋值
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

对于`DataFrame`，`[]`可以返回`DataFrame`对象的一列或多列。

In [23]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [24]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [25]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [26]:
# 返回行，这是一种简化了的写法，需要重点注意
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [27]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


根据筛选条件返回`DataFrame`对应布尔值矩阵。

In [28]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [31]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### loc与iloc

对于`DataFrame`的行索引标签，可以使用`loc`和`iloc`操作符，用于选择`DataFrame`对象行与列的子集。

`loc`操作符接受行列`index`或`columns`标签值作为参数，`iloc`操作符接受行与列的计数值作为参数。

    loc[row_label, column_label]
    iloc[row_index, column_index]
    at[row_label, column_label]
    iat[row_index, column_index]

In [36]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [32]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [33]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [34]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [35]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [37]:
# slice
data.loc[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [38]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


## 整数索引

当`Series`或`DataFrame`中的`index`或`columns`以给定的索引值为整数时，在数据选择时会出现歧义。

In [39]:
ser = pd.Series(np.arange(3.0))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [41]:
ser2 = pd.Series(np.arange(3.0), index=['a', 'b', 'c'])
ser2

a    0.0
b    1.0
c    2.0
dtype: float64

In [42]:
# 对以整数作为index的Series存在歧义，出现错误
# ser[-1]

# 对以非整数作为index的Series不存在歧义
ser2[-1]

2.0

当`Series`的索引为整数时，为保证索引行为的一致性，若使用整数作为`index`或`columns`，则数据选择以索引值为准

其中`loc`针对索引标签选择操作，而`iloc`针对整数索引选择操作。

In [43]:
ser[:1]

0    0.0
dtype: float64

In [44]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [45]:
ser.iloc[:1]

0    0.0
dtype: float64

## 代数与数据对齐

### Series对象

In [47]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [48]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [49]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

### DataFrame

In [50]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [51]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [52]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


对没有相同行列索引的`DataFrame`求和操作，结果`DataFrame`各元素均为`NaN`。

In [53]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


### 算数计算中的空值填充

In [54]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [55]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [56]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0
